In [1]:
# Author: Gergely Zahoranszky-Kohalmi, PhD
#
#
# Email: gergely.zahoranszky-kohalmi@nih.gov
#
# Organization: National Center for Advancing Translational Sciences (NCATS/NIH)
#
#

In [2]:
from hilbertcurve.hilbertcurve import HilbertCurve

import numpy as np

#from hilbertcurve.hilbertcurve import HilbertCurve

import networkx as nx

import math
import json

In [3]:

def get_hilbert_coordinates (hc, bucket_id):
#    print (bucket_id)
    coordinates = []
    coordinates = hc.coordinates_from_distance(bucket_id - 1)
    coordinate_str = ''
    nr_dim = len (coordinates)

    for i in range(nr_dim):
        coordinate_str += str(coordinates[i]) + ';'

    coordinate_str = coordinate_str[:-1]

    return (coordinate_str)

def get_conflicting_neighbors (x, y, max_coord, points, diff):
    conflicting_neighbors = []
    for i in range (max(0, x - 1), min(max_coord, x + 1) + 1):
        for j in range (max(0, y - 1), min(max_coord, y + 1) + 1):
            #print (f'(x: {x}, y: {y}) - (n_i: {i}, n_j: {j}')

            point_a = points[x][y]
            point_b = points[i][j]
            
            
            if (abs(point_a - point_b) > diff) and (point_a > point_b):    # we're creating an upper triangular adjacency matrix
                conflicting_neighbors.append ((point_a, point_b))          # so keeping only this edge is ok, since the relation is symmetric
                                                                           # this also takes care of the x!=i AND y!=j case

    #print (len(conflicting_neighbors))
    
    return (conflicting_neighbors)
                
def to_cyjson(G):
    graph_json = (nx.cytoscape_data(G, name='idx', ident='node_id'))
    

    
    graph_json =  graph_json.replace('\'', '"').replace('True', 'true').replace('False', 'false')
      
    return (graph_json)
    

In [4]:
min_coord = 0
max_coord = 255

min_bucket_id = 1

max_bucket_id = int((max_coord + 1) * (max_coord + 1))
n_dim = 2
hc_order = 8

#diff_cutoff = 0.5
#diff_cutoff = 0.25
diff_cutoff = 0.05

diff = math.floor(max_bucket_id * diff_cutoff)
print (diff)

points = np.zeros(((max_coord + 1), (max_coord + 1)), dtype = int)


hc = HilbertCurve(hc_order, n_dim)

x = -1
y = -1

tmp = []

points_coord = {}

for i in range (min_bucket_id, max_bucket_id + 1):
    
    point = get_hilbert_coordinates (hc, i)
    #print (i, point)
    
    tmp = point.split (';')
    
    x = int (tmp[0])
    y = int (tmp[1])

    points[x][y] = i
    
    points_coord[i] = (x, y)

print (points[:5])


3276
[[    1     4     5 ... 21844 21845 21846]
 [    2     3     8 ... 21843 21848 21847]
 [   15    14     9 ... 21854 21849 21850]
 [   16    13    12 ... 21853 21852 21851]
 [   17    18    31 ... 21860 21861 21862]]


In [5]:
edges = []
neighbors = []

for i in range (max_coord + 1):
    for j in range (max_coord + 1):
        neighbors = []    
        neighbors = get_conflicting_neighbors (i, j, max_coord, points, diff)
        if len (neighbors) > 0:
            edges.extend(neighbors)

print (len(edges))

print (edges)


3270
[(15019, 1366), (15019, 1367), (20139, 16726), (20139, 16727), (15018, 1366), (15018, 1367), (15018, 1370), (20138, 16726), (20138, 16727), (20138, 16730), (15015, 1367), (15015, 1370), (15015, 1371), (20135, 16727), (20135, 16730), (20135, 16731), (15014, 1370), (15014, 1371), (15014, 1382), (20134, 16730), (20134, 16731), (20134, 16742), (15003, 1371), (15003, 1382), (15003, 1383), (20123, 16731), (20123, 16742), (20123, 16743), (15002, 1382), (15002, 1383), (15002, 1386), (20122, 16742), (20122, 16743), (20122, 16746), (14999, 1383), (14999, 1386), (14999, 1387), (20119, 16743), (20119, 16746), (20119, 16747), (14998, 1386), (14998, 1387), (14998, 1430), (20118, 16746), (20118, 16747), (20118, 16790), (14955, 1387), (14955, 1430), (14955, 1431), (20075, 16747), (20075, 16790), (20075, 16791), (14954, 1430), (14954, 1431), (14954, 1434), (20074, 16790), (20074, 16791), (20074, 16794), (14951, 1431), (14951, 1434), (14951, 1435), (20071, 16791), (20071, 16794), (14950, 1434), (14

In [6]:
G = nx.Graph()


idx = 0

G.add_node(1)
G.nodes[1]['x'] = 0
G.nodes[1]['y'] = 0
G.nodes[1]['idx'] = idx
G.nodes[1]['node_id'] = 0

idx += 1

G.add_node (max_bucket_id)
G.nodes[max_bucket_id]['x'] = max_coord - 1
G.nodes[max_bucket_id]['y'] = max_coord - 1
G.nodes[max_bucket_id]['idx'] = idx
G.nodes[max_bucket_id]['node_id'] = max_bucket_id

idx += 1

unique_nodes = set()

for edge in edges:
    #print (edge[0])
    unique_nodes.add (edge[0])
    unique_nodes.add (edge[1])

unique_nodes = list(unique_nodes)

#print (unique_nodes)

for n in unique_nodes:
    G.add_node(n)
    #print (n)
    #print (points_coord[n][0])
    G.nodes[n]['x'] = points_coord[n][0]
    G.nodes[n]['y'] = points_coord[n][1]
    G.nodes[n]['idx'] = idx
    G.nodes[n]['node_id'] = n

    idx += 1

for edge in edges:
    G.add_edge(edge[0], edge[1])

    
#print (G.nodes(data = True))
#graph_json = to_cyjson (G)
        
        
#print (graph_json)
#G.gra

#nx.readwrite.graphml.write_graphml
nx.write_graphml(G, 'canyon.graphml')

In [7]:
# References


# Ref: https://stackoverflow.com/questions/58422957/how-can-i-create-a-2d-array-of-integers
# Ref: https://geeksforgeeks.org/abs-in-python/
# Ref: https://networkx.org/documentation/stable/reference/readwrite/generated/networkx.readwrite.json_graph.cytoscape_data.html#networkx.readwrite.json_graph.cytoscape_data
